In [ ]:
#ipcluster start -n=8

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import time
import numexpr as ne
import seaborn as sns
from ipywidgets import interact
from sklearn.decomposition import PCA
from numba import jit

In [2]:
ne.set_num_threads(32)

8

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
%gui qt

In [5]:
import sys
sys.path.append('../../..')

In [6]:
from spiketag.mvc import Sorter
from spiketag.base import ProbeFactory, MUA, bload, SPKTAG
from spiketag.utils import conf

# Test 32 liner probe

In [ ]:
nCh = 32
fs = 25000
probe = ProbeFactory.genLinearProbe(fs=fs, n_ch=nCh)
probe

In [ ]:
app = Sorter('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie_32_25k.bin',probe = probe, 
              corr_cutoff=0.95, 
              fetlen=6, 
              fet_whiten=False,
              fall_off_size=18, 
              n_jobs=1)
# app = Sorter('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie_32_25k.bin', 
#              spktag_filename='/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie_32_25k.bin.probe')



In [ ]:
#app.model.spktag.tofile('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie_32_25k.bin.probe')

In [14]:
app.run()

[VIEW] Spikeview -- render - step 0: get data, elapsed time: 86.9450569153 ms
[VIEW] Spikeview -- render - step 1: set data, elapsed time: 82.7651023865 ms
[VIEW] Spikeview -- render - step 2: gsgl update, elapsed time: 0.351905822754 ms
[View] Ctreeview -- get_cluster_by_pos., elapsed time: 1.84202194214 ms
[View] Ctreeview -- get_cluster_by_pos., elapsed time: 1.79195404053 ms
[View] Ctreeview -- get_cluster_by_pos., elapsed time: 2.30407714844 ms
[View] Ctreeview -- get_cluster_by_pos., elapsed time: 1.67202949524 ms
[View] Ctreeview -- get_cluster_by_pos., elapsed time: 1.66606903076 ms
[View] Ctreeview -- get_cluster_by_pos., elapsed time: 1.6610622406 ms
[View] Ctreeview -- get_cluster_by_pos., elapsed time: 1.62410736084 ms
[View] Ctreeview -- get_cluster_by_pos., elapsed time: 1.61981582642 ms
[View] Ctreeview -- get_cluster_by_pos., elapsed time: 1.74307823181 ms
[View] Ctreeview -- get_cluster_by_pos., elapsed time: 1.7249584198 ms
[View] Ctreeview -- get_cluster_by_pos., ela

# Switch of profiler log, if True, print profiler log

In [7]:
from spiketag.utils import conf

In [8]:
conf.ENABLE_PROFILER = True

# Use line profiler to find bottomneck

In [ ]:
app.model.clu[30].selectlist

In [ ]:
app.model.clu[30].index

In [ ]:
# pip install line-profiler first.
%load_ext line_profiler

In [ ]:
%lprun -f app.view.trace_view.locate_and_highlight app.view.trace_view.locate_and_highlight(app.view.spk_view.selected_spk)

# Test 160 tetrode probe

In [7]:
nCh = 64
fs = 25e3
tetrode = ProbeFactory.genTetrodeProbe(fs=fs, n_ch=nCh)
tetrode.reorder_by_chip = False
tetrode

0:[0 1 2 3]
1:[4 5 6 7]
2:[ 8  9 10 11]
3:[12 13 14 15]
4:[16 17 18 19]
5:[20 21 22 23]
6:[24 25 26 27]
7:[28 29 30 31]
8:[32 33 34 35]
9:[36 37 38 39]
10:[40 41 42 43]
11:[44 45 46 47]
12:[48 49 50 51]
13:[52 53 54 55]
14:[56 57 58 59]
15:[60 61 62 63]

In [8]:
app = Sorter('/Users/chiy/Documents/HHMI/Github/spiketag_test_data/ttd_64_25k_20170712_without_noise.bin',probe = tetrode, 
              corr_cutoff=0.95, 
              fet_method='pca',
              fetlen=6, 
              fet_whiten=False,
              fall_off_size=18, 
              spklen=19,
              n_jobs=1)
#app = Sorter('/Users/chiy/Documents/HHMI/Github/spiketag_test_data/ttd_64_25k_20170712_without_noise.bin', spktag_filename='/Users/chiy/Documents/HHMI/Github/spiketag_test_data/ttd_64_25k_20170712_without_noise_spktag.bin')

2017-08-09 11:25:38,695 - spiketag - INFO - load mua data
2017-08-09 11:25:38,716 - spiketag - INFO - #############  load data  ###################
2017-08-09 11:25:38,718 - spiketag - INFO - /Users/chiy/Documents/HHMI/Github/spiketag_test_data/ttd_64_25k_20170712_without_noise.bin loaded, it contains: 
2017-08-09 11:25:38,719 - spiketag - INFO - 7361618 * 64 points (1884574208 bytes) 
2017-08-09 11:25:38,720 - spiketag - INFO - 64 channels with sampling rate of 25000.0000 
2017-08-09 11:25:38,721 - spiketag - INFO - 294.465 secs (4.908 mins) of data
2017-08-09 11:25:38,722 - spiketag - INFO - #############################################
2017-08-09 11:25:41,068 - spiketag - INFO - raw data have 74653 spks
2017-08-09 11:25:41,069 - spiketag - INFO - removing all spks on group which len(spks) less then fetlen
2017-08-09 11:25:41,077 - spiketag - INFO - removed all spks on these groups: {0: 1, 9: 2, 2: 4}
2017-08-09 11:25:41,078 - spiketag - INFO - grouping spike time
2017-08-09 11:25:41

In [9]:
app.run()

2017-08-09 11:25:52,168 - spiketag - WARNING -  group 0 has no spikes! 
2017-08-09 11:26:02,548 - spiketag - WARNING -  group 2 has no spikes! 


[View] Ctreeview -- select cluster 142, elapsed time: 10.0400447845 ms
[View] Ctreeview -- select cluster 143, elapsed time: 10.3540420532 ms
[View] Ctreeview -- select cluster 141, elapsed time: 1.71399116516 ms
[View] Ctreeview -- select cluster 141, elapsed time: 1.5709400177 ms
[View] Ctreeview -- select cluster 15213, elapsed time: 2.43210792542 ms
[View] Ctreeview -- select cluster 15214, elapsed time: 2.01106071472 ms
[View] Ctreeview -- select cluster 15210, elapsed time: 95.3509807587 ms
[View] Ctreeview -- select cluster 15214, elapsed time: 18.7690258026 ms
[View] Ctreeview -- select cluster 15215, elapsed time: 2.32100486755 ms
[View] Ctreeview -- select cluster 15222, elapsed time: 14.1739845276 ms
[View] Ctreeview -- select cluster 15222, elapsed time: 19.6039676666 ms
[View] Ctreeview -- select cluster 15226, elapsed time: 10.9870433807 ms
[View] Ctreeview -- select cluster 15225, elapsed time: 5.58495521545 ms
[View] Ctreeview -- select cluster 15222, elapsed time: 14.5

2017-08-09 11:31:46,385 - spiketag - WARNING -  group 9 has no spikes! 


[View] Ctreeview -- select cluster 7453, elapsed time: 17.8949832916 ms
[View] Ctreeview -- select cluster 7455, elapsed time: 7.52687454224 ms
[View] Ctreeview -- select cluster 7455, elapsed time: 1.51395797729 ms
[View] Ctreeview -- select cluster 7453, elapsed time: 12.2811794281 ms
[View] Ctreeview -- select cluster 7453, elapsed time: 19.2000865936 ms
[View] Ctreeview -- select cluster 7453, elapsed time: 20.8361148834 ms
[View] Ctreeview -- select cluster 7452, elapsed time: 10.712146759 ms
[View] Ctreeview -- select cluster 7453, elapsed time: 11.3799571991 ms
[View] Ctreeview -- select cluster 7453, elapsed time: 20.8110809326 ms
[View] Ctreeview -- select cluster 7451, elapsed time: 29.4260978699 ms
[View] Ctreeview -- select cluster 7452, elapsed time: 7.58790969849 ms
[View] Ctreeview -- select cluster 7453, elapsed time: 13.1411552429 ms
[View] Ctreeview -- select cluster 7453, elapsed time: 18.2211399078 ms
[View] Ctreeview -- select cluster 7454, elapsed time: 13.7100219

[View] Ctreeview -- select cluster 7454, elapsed time: 9.82880592346 ms
[View] Ctreeview -- select cluster 7453, elapsed time: 19.1309452057 ms
[View] Ctreeview -- select cluster 7458, elapsed time: 10.5950832367 ms
[View] Ctreeview -- select cluster 7454, elapsed time: 8.26287269592 ms
[View] Ctreeview -- select cluster 7456, elapsed time: 8.76998901367 ms
[View] Ctreeview -- select cluster 7458, elapsed time: 5.93304634094 ms
[View] Ctreeview -- select cluster 7458, elapsed time: 4.88018989563 ms
[View] Ctreeview -- select cluster 7458, elapsed time: 4.5690536499 ms
[View] Ctreeview -- select cluster 7457, elapsed time: 3.71694564819 ms
[View] Ctreeview -- select cluster 7457, elapsed time: 2.23588943481 ms
[View] Ctreeview -- select cluster 7458, elapsed time: 3.44204902649 ms
[View] Ctreeview -- select cluster 7458, elapsed time: 4.57501411438 ms
[View] Ctreeview -- select cluster 7457, elapsed time: 3.56602668762 ms
[View] Ctreeview -- select cluster 7457, elapsed time: 2.19011306

[View] Ctreeview -- select cluster 9701, elapsed time: 3.36003303528 ms
[View] Ctreeview -- select cluster 9699, elapsed time: 3.85618209839 ms
[View] Ctreeview -- select cluster 9699, elapsed time: 5.76400756836 ms
[View] Ctreeview -- select cluster 9701, elapsed time: 3.45301628113 ms
[View] Ctreeview -- select cluster 9701, elapsed time: 1.56497955322 ms
[View] Ctreeview -- select cluster 9701, elapsed time: 1.59192085266 ms
[View] Ctreeview -- select cluster 9700, elapsed time: 3.72505187988 ms
[View] Ctreeview -- select cluster 9701, elapsed time: 3.20100784302 ms
[View] Ctreeview -- select cluster 9701, elapsed time: 1.58405303955 ms
[View] Ctreeview -- select cluster 9700, elapsed time: 3.84902954102 ms
[View] Ctreeview -- select cluster 9701, elapsed time: 3.31807136536 ms
[View] Ctreeview -- select cluster 9701, elapsed time: 1.56903266907 ms
[View] Ctreeview -- select cluster 9699, elapsed time: 3.80706787109 ms
[View] Ctreeview -- select cluster 9699, elapsed time: 5.6760311

[View] Ctreeview -- select cluster 9695, elapsed time: 1.86109542847 ms
[View] Ctreeview -- select cluster 9695, elapsed time: 1.49083137512 ms
[View] Ctreeview -- select cluster 9694, elapsed time: 8.72397422791 ms
[View] Ctreeview -- select cluster 9694, elapsed time: 12.9950046539 ms
[View] Ctreeview -- select cluster 9695, elapsed time: 6.29711151123 ms
[View] Ctreeview -- select cluster 9695, elapsed time: 1.64604187012 ms
[View] Ctreeview -- select cluster 9692, elapsed time: 14.6210193634 ms
[View] Ctreeview -- select cluster 9695, elapsed time: 8.49413871765 ms
[View] Ctreeview -- select cluster 9692, elapsed time: 14.4901275635 ms
[View] Ctreeview -- select cluster 9692, elapsed time: 20.4229354858 ms
[View] Ctreeview -- select cluster 9694, elapsed time: 16.6280269623 ms
[View] Ctreeview -- select cluster 9695, elapsed time: 6.10113143921 ms
[View] Ctreeview -- select cluster 9692, elapsed time: 14.6188735962 ms
[View] Ctreeview -- select cluster 9694, elapsed time: 15.513896

[View] Ctreeview -- select cluster 17578, elapsed time: 15.0871276855 ms
[View] Ctreeview -- select cluster 17581, elapsed time: 17.4980163574 ms
[View] Ctreeview -- select cluster 17578, elapsed time: 16.5729522705 ms
[View] Ctreeview -- select cluster 17581, elapsed time: 15.6519412994 ms
[View] Ctreeview -- select cluster 17581, elapsed time: 14.2068862915 ms
[View] Ctreeview -- select cluster 17578, elapsed time: 17.2281265259 ms
[View] Ctreeview -- select cluster 17581, elapsed time: 14.4808292389 ms
[View] Ctreeview -- select cluster 17581, elapsed time: 12.4890804291 ms
[View] Ctreeview -- select cluster 17581, elapsed time: 12.5210285187 ms
[View] Ctreeview -- select cluster 17580, elapsed time: 6.90793991089 ms
[View] Ctreeview -- select cluster 17580, elapsed time: 1.93405151367 ms
[View] Ctreeview -- select cluster 17580, elapsed time: 1.97100639343 ms
[View] Ctreeview -- select cluster 17580, elapsed time: 1.85608863831 ms
[View] Ctreeview -- select cluster 17580, elapsed t

In [14]:
app.view.ctree_view.set_data(app.model.clu[3])

received mouse press event
received mouse press button == 1
current_cluster is None
[View] Ctreeview -- select cluster 142, elapsed time: 16.8590545654 ms
[View] Ctreeview -- select cluster 142, elapsed time: 29.4728279114 ms
[View] Ctreeview -- select cluster 143, elapsed time: 1.04594230652 ms
[View] Ctreeview -- select cluster 141, elapsed time: 1.5881061554 ms
[View] Ctreeview -- select cluster 141, elapsed time: 1.67298316956 ms
[View] Ctreeview -- select cluster 142, elapsed time: 1.00779533386 ms
[View] Ctreeview -- select cluster 142, elapsed time: 0.848054885864 ms
[View] Ctreeview -- select cluster 142, elapsed time: 0.971078872681 ms
[View] Ctreeview -- select cluster 141, elapsed time: 1.45697593689 ms
[View] Ctreeview -- select cluster 141, elapsed time: 1.79195404053 ms
[View] Ctreeview -- select cluster 143, elapsed time: 1.12795829773 ms
[View] Ctreeview -- select cluster 142, elapsed time: 0.955820083618 ms
[View] Ctreeview -- select cluster 142, elapsed time: 0.908851

2017-08-08 13:55:57,474 - spiketag - WARNING -  group 9 has no spikes! 


[View] Ctreeview -- select cluster 9692, elapsed time: 20.763874054 ms
[View] Ctreeview -- select cluster 9700, elapsed time: 12.0828151703 ms
[View] Ctreeview -- select cluster 9700, elapsed time: 5.41210174561 ms
[View] Ctreeview -- select cluster 9698, elapsed time: 3.87907028198 ms
[View] Ctreeview -- select cluster 9694, elapsed time: 9.81903076172 ms
[View] Ctreeview -- select cluster 9697, elapsed time: 13.1008625031 ms
[View] Ctreeview -- select cluster 9700, elapsed time: 7.29608535767 ms
[View] Ctreeview -- select cluster 9700, elapsed time: 5.72109222412 ms
[View] Ctreeview -- select cluster 9698, elapsed time: 3.58891487122 ms
[View] Ctreeview -- select cluster 9700, elapsed time: 4.03094291687 ms
[View] Ctreeview -- select cluster 9701, elapsed time: 3.26085090637 ms
[View] Ctreeview -- select cluster 9695, elapsed time: 1.45506858826 ms
[View] Ctreeview -- select cluster 9691, elapsed time: 38.0289554596 ms
[View] Ctreeview -- select cluster 17564, elapsed time: 2.5069713

received mouse press event
[View] Ctreeview -- select cluster 17585, elapsed time: 5.31315803528 ms
[View] Ctreeview -- select cluster 17579, elapsed time: 4.66084480286 ms
received mouse press event
[View] Ctreeview -- select cluster 17578, elapsed time: 17.7118778229 ms
received mouse press event
received mouse press button == 1
current_cluster is None
[View] Ctreeview -- select cluster 17580, elapsed time: 9.05203819275 ms
received mouse press event
[View] Ctreeview -- select cluster 17578, elapsed time: 12.4790668488 ms
[View] Ctreeview -- select cluster 17581, elapsed time: 16.664981842 ms
[View] Ctreeview -- select cluster 17578, elapsed time: 17.6768302917 ms
[View] Ctreeview -- select cluster 17577, elapsed time: 25.9070396423 ms
[View] Ctreeview -- select cluster 17577, elapsed time: 22.8841304779 ms
[View] Ctreeview -- select cluster 17578, elapsed time: 18.6080932617 ms
[View] Ctreeview -- select cluster 17574, elapsed time: 9.48309898376 ms
[View] Ctreeview -- select cluste

  File "/Users/chiy/anaconda/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Users/chiy/anaconda/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/chiy/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/chiy/anaconda/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/chiy/anaconda/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/Users/chiy/anaconda/lib/python2.7/site-packages/zmq/eventloop/ioloop.py", line 162, in start
    super(ZMQIOLoop, self).start()
  File "/Users/chiy/anaconda/lib/python2.7/site-packages/tornado/ioloop.py", line 831, in start
    self._run_callback(callback)
  File "/Users/chiy/anaconda/lib/python2.7/site-packages/tornado/ioloop.py", line 604, in _run_c

...

received mouse press event
received mouse press button == 1
current_cluster is 17566
collapse 17566 here
children is []
collapse from [17562 17563]
[View] Ctreeview -- select cluster 17567, elapsed time: 37.397146225 ms
[View] Ctreeview -- select cluster 17566, elapsed time: 15.6779289246 ms
[View] Ctreeview -- select cluster 17565, elapsed time: 42.6850318909 ms
received mouse press event
received mouse press button == 1
current_cluster is 17565
collapse 17565 here
received mouse press event
received mouse press button == 1
current_cluster is 17565
collapse 17565 here
received mouse press event
received mouse press button == 1
current_cluster is None
received mouse press event
[View] Ctreeview -- select cluster 17565, elapsed time: 48.1150150299 ms
received mouse press event
received mouse press button == 1
current_cluster is None
[View] Ctreeview -- select cluster 17565, elapsed time: 46.5569496155 ms
[View] Ctreeview -- select cluster 17562, elapsed time: 59.0560436249 ms
received m

  File "/Users/chiy/anaconda/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Users/chiy/anaconda/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/chiy/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/chiy/anaconda/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/chiy/anaconda/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/Users/chiy/anaconda/lib/python2.7/site-packages/zmq/eventloop/ioloop.py", line 162, in start
    super(ZMQIOLoop, self).start()
  File "/Users/chiy/anaconda/lib/python2.7/site-packages/tornado/ioloop.py", line 831, in start
    self._run_callback(callback)
  File "/Users/chiy/anaconda/lib/python2.7/site-packages/tornado/ioloop.py", line 604, in _run_c

received mouse press event
received mouse press button == 1
current_cluster is 17566
collapse 17566 here
children is []
collapse from [17563 17564 17565]
[View] Ctreeview -- select cluster 17561, elapsed time: 58.7029457092 ms
[View] Ctreeview -- select cluster 17567, elapsed time: 41.5508747101 ms
[View] Ctreeview -- select cluster 17565, elapsed time: 38.9349460602 ms
[View] Ctreeview -- select cluster 17565, elapsed time: 39.6611690521 ms
[View] Ctreeview -- select cluster 17566, elapsed time: 14.111995697 ms
[View] Ctreeview -- select cluster 17566, elapsed time: 2.45690345764 ms
[View] Ctreeview -- select cluster 17565, elapsed time: 28.9499759674 ms
received mouse press event
[View] Ctreeview -- select cluster 17565, elapsed time: 46.1099147797 ms
[View] Ctreeview -- select cluster 17565, elapsed time: 50.2741336823 ms
[View] Ctreeview -- select cluster 17565, elapsed time: 47.6310253143 ms
[View] Ctreeview -- select cluster 17566, elapsed time: 14.6789550781 ms
[View] Ctreeview 

  File "/Users/chiy/anaconda/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Users/chiy/anaconda/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/chiy/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/chiy/anaconda/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/chiy/anaconda/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/Users/chiy/anaconda/lib/python2.7/site-packages/zmq/eventloop/ioloop.py", line 162, in start
    super(ZMQIOLoop, self).start()
  File "/Users/chiy/anaconda/lib/python2.7/site-packages/tornado/ioloop.py", line 831, in start
    self._run_callback(callback)
  File "/Users/chiy/anaconda/lib/python2.7/site-packages/tornado/ioloop.py", line 604, in _run_c

received mouse press event
received mouse press button == 1
current_cluster is 17567
collapse 17567 here
children is [17568 17569]
collapse from [17563 17564 17566 17568 17569]
collapse to [17563 17564 17566 17567]
[View] Ctreeview -- select cluster 17565, elapsed time: 60.2788925171 ms
received mouse press event
received mouse press event
[View] Ctreeview -- select cluster 17571, elapsed time: 38.7368202209 ms
received mouse press event
received mouse press button == 1
current_cluster is None
received mouse press event
[View] Ctreeview -- select cluster 17569, elapsed time: 37.4159812927 ms
[View] Ctreeview -- select cluster 17569, elapsed time: 38.1579399109 ms
[View] Ctreeview -- select cluster 17569, elapsed time: 36.2048149109 ms
[View] Ctreeview -- select cluster 17569, elapsed time: 36.838054657 ms
received mouse press event
received mouse press button == 1
current_cluster is None
received mouse press event
received mouse press button == 1
current_cluster is None
received mouse 

In [12]:
app.view.ctree_view._clu_tree.shape

(8,)

[View] Ctreeview -- select cluster 6459, elapsed time: 4.802942276 ms
[View] Ctreeview -- select cluster 6458, elapsed time: 17.6639556885 ms
[View] Ctreeview -- select cluster 6460, elapsed time: 19.4978713989 ms
[View] Ctreeview -- select cluster 6463, elapsed time: 7.04002380371 ms
[View] Ctreeview -- select cluster 6465, elapsed time: 7.4508190155 ms
[View] Ctreeview -- select cluster 6464, elapsed time: 11.5540027618 ms
[View] Ctreeview -- select cluster 6460, elapsed time: 16.9379711151 ms
[View] Ctreeview -- select cluster 6463, elapsed time: 7.29513168335 ms
[View] Ctreeview -- select cluster 6465, elapsed time: 6.5221786499 ms
[View] Ctreeview -- select cluster 6458, elapsed time: 21.3449001312 ms
[View] Ctreeview -- select cluster 6458, elapsed time: 25.0079631805 ms
[View] Ctreeview -- select cluster 6459, elapsed time: 7.65013694763 ms
[View] Ctreeview -- select cluster 6460, elapsed time: 13.4661197662 ms
[View] Ctreeview -- select cluster 6460, elapsed time: 18.5599327087

2017-08-09 12:14:26,233 - spiketag - INFO - received model modified event, removed spikes[group=14, global_ids=[343]]
  File "/Users/chiy/anaconda/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Users/chiy/anaconda/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/chiy/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/chiy/anaconda/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/chiy/anaconda/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/Users/chiy/anaconda/lib/python2.7/site-packages/zmq/eventloop/ioloop.py", line 162, in start
    super(ZMQIOLoop, self).start()
  File "/Users/chiy/anaconda/lib/python2.7/site-packages/tornado/ioloop.py", line 831, in start
    self._r

[View] Ctreeview -- select cluster 11077, elapsed time: 48.9299297333 ms
[View] Ctreeview -- select cluster 11077, elapsed time: 46.2191104889 ms
[View] Ctreeview -- select cluster 11077, elapsed time: 45.4349517822 ms
[View] Ctreeview -- select cluster 11077, elapsed time: 47.5180149078 ms
[View] Ctreeview -- select cluster 11078, elapsed time: 15.1369571686 ms
[View] Ctreeview -- select cluster 11078, elapsed time: 5.79309463501 ms
[View] Ctreeview -- select cluster 11077, elapsed time: 36.3500118256 ms
[View] Ctreeview -- select cluster 11078, elapsed time: 5.42116165161 ms
[View] Ctreeview -- select cluster 11080, elapsed time: 2.89487838745 ms
[View] Ctreeview -- select cluster 11082, elapsed time: 2.11882591248 ms
[View] Ctreeview -- select cluster 11081, elapsed time: 3.0529499054 ms
[View] Ctreeview -- select cluster 11080, elapsed time: 2.690076828 ms
[View] Ctreeview -- select cluster 11081, elapsed time: 2.77209281921 ms
[View] Ctreeview -- select cluster 11077, elapsed time

[View] Ctreeview -- select cluster 11078, elapsed time: 4.1868686676 ms
[View] Ctreeview -- select cluster 11078, elapsed time: 5.27811050415 ms
[View] Ctreeview -- select cluster 11080, elapsed time: 3.12900543213 ms
[View] Ctreeview -- select cluster 11080, elapsed time: 2.25591659546 ms
[View] Ctreeview -- select cluster 11080, elapsed time: 2.35986709595 ms
[View] Ctreeview -- select cluster 11080, elapsed time: 2.16293334961 ms
[View] Ctreeview -- select cluster 11080, elapsed time: 2.2120475769 ms
[View] Ctreeview -- select cluster 11080, elapsed time: 2.25210189819 ms
[View] Ctreeview -- select cluster 11078, elapsed time: 4.16994094849 ms
[View] Ctreeview -- select cluster 11080, elapsed time: 3.2970905304 ms
[View] Ctreeview -- select cluster 11080, elapsed time: 2.38990783691 ms
[View] Ctreeview -- select cluster 11078, elapsed time: 4.17613983154 ms
[View] Ctreeview -- select cluster 11082, elapsed time: 2.97617912292 ms
[View] Ctreeview -- select cluster 11083, elapsed time

...

[View] Ctreeview -- select cluster 9697, elapsed time: 6.09993934631 ms
[View] Ctreeview -- select cluster 9697, elapsed time: 8.54897499084 ms
[View] Ctreeview -- select cluster 9701, elapsed time: 4.54616546631 ms
[View] Ctreeview -- select cluster 9701, elapsed time: 1.51014328003 ms
[View] Ctreeview -- select cluster 9700, elapsed time: 4.30297851562 ms
[View] Ctreeview -- select cluster 9700, elapsed time: 5.37705421448 ms
[View] Ctreeview -- select cluster 9700, elapsed time: 5.52010536194 ms
[View] Ctreeview -- select cluster 9697, elapsed time: 7.39002227783 ms


2017-08-09 12:45:47,013 - spiketag - DEBUG - ctree view collapse cluster 9697 here


[View] Ctreeview -- select cluster 9697, elapsed time: 8.43811035156 ms
[View] Ctreeview -- select cluster 9692, elapsed time: 20.1759338379 ms
[View] Ctreeview -- select cluster 9694, elapsed time: 16.9970989227 ms
[View] Ctreeview -- select cluster 9694, elapsed time: 15.2349472046 ms
[View] Ctreeview -- select cluster 9695, elapsed time: 6.11615180969 ms
[View] Ctreeview -- select cluster 9695, elapsed time: 1.55401229858 ms
[View] Ctreeview -- select cluster 9694, elapsed time: 11.0168457031 ms
[View] Ctreeview -- select cluster 9694, elapsed time: 14.4939422607 ms
[View] Ctreeview -- select cluster 9692, elapsed time: 21.1238861084 ms
[View] Ctreeview -- select cluster 9694, elapsed time: 17.4009799957 ms
[View] Ctreeview -- select cluster 9694, elapsed time: 15.4888629913 ms
[View] Ctreeview -- select cluster 9695, elapsed time: 6.04104995728 ms
[View] Ctreeview -- select cluster 9695, elapsed time: 1.70612335205 ms
[View] Ctreeview -- select cluster 9695, elapsed time: 1.6450881

[View] Ctreeview -- select cluster 9698, elapsed time: 3.39388847351 ms
[View] Ctreeview -- select cluster 9698, elapsed time: 1.92499160767 ms
[View] Ctreeview -- select cluster 9699, elapsed time: 3.84402275085 ms
[View] Ctreeview -- select cluster 9697, elapsed time: 6.50501251221 ms
[View] Ctreeview -- select cluster 9699, elapsed time: 5.69987297058 ms
[View] Ctreeview -- select cluster 9697, elapsed time: 6.24895095825 ms
[View] Ctreeview -- select cluster 9699, elapsed time: 6.73913955688 ms
[View] Ctreeview -- select cluster 9699, elapsed time: 4.91809844971 ms
[View] Ctreeview -- select cluster 9697, elapsed time: 6.25705718994 ms
[View] Ctreeview -- select cluster 9699, elapsed time: 5.72609901428 ms
[View] Ctreeview -- select cluster 9697, elapsed time: 7.51805305481 ms
[View] Ctreeview -- select cluster 9697, elapsed time: 7.2169303894 ms
[View] Ctreeview -- select cluster 9699, elapsed time: 5.71584701538 ms
[View] Ctreeview -- select cluster 9699, elapsed time: 6.32405281

[View] Ctreeview -- select cluster 9699, elapsed time: 5.66911697388 ms
[View] Ctreeview -- select cluster 9699, elapsed time: 5.6791305542 ms
[View] Ctreeview -- select cluster 9699, elapsed time: 6.34098052979 ms
[View] Ctreeview -- select cluster 9699, elapsed time: 5.62286376953 ms
[View] Ctreeview -- select cluster 9701, elapsed time: 3.81517410278 ms
[View] Ctreeview -- select cluster 9701, elapsed time: 1.58381462097 ms
[View] Ctreeview -- select cluster 9699, elapsed time: 4.01186943054 ms
[View] Ctreeview -- select cluster 9699, elapsed time: 5.72395324707 ms
[View] Ctreeview -- select cluster 9700, elapsed time: 5.22899627686 ms
[View] Ctreeview -- select cluster 9700, elapsed time: 5.08809089661 ms
[View] Ctreeview -- select cluster 9699, elapsed time: 5.11193275452 ms
[View] Ctreeview -- select cluster 9699, elapsed time: 5.18989562988 ms
[View] Ctreeview -- select cluster 9700, elapsed time: 5.47003746033 ms
[View] Ctreeview -- select cluster 9699, elapsed time: 5.51009178

In [ ]:
views = [app.view.ctree_view, app.view.spk_view, app.view.amplitude_view, app.view.raster_view, app.view.scatter_view]

In [ ]:
func = 'isWidgetType'
for view in views:
    print '{} of {} is {}'.format(func, view.name, getattr(view._backend, func)())

In [ ]:
None == None

In [ ]:
dir(app.view.gui.get_view('amplitude_view0')._backend)

In [ ]:
app.view.gui.get_view('spike_view0')._backend.backgroundRole()

In [ ]:
app.view.amplitude_view

In [14]:
%time app.view.ctree_view._get_leaves(app.view.ctree_view._whole_tree,17561)

CPU times: user 726 ms, sys: 12.5 ms, total: 738 ms
Wall time: 738 ms


array([4697, 4704, 1805, ..., 6321, 1515, 2736])

In [13]:
app.view.ctree_view._get_leaves(app.view.ctree_view._clu_tree)

array([], dtype=int64)

In [22]:
app.view.ctree_view._whole_tree.shape

(17587,)

In [17]:
def _recurse_leaf_dfs(tree, root):
    children = tree[tree['parent'] == root]['child']
    if len(children) == 0:
        return [root, ]
    else:
        return sum([_recurse_leaf_dfs(tree, child) for child in children], [])
         
    
def _get_leaves(tree,root):
  #  tree = tree[tree['child_size'] > 1]
   # root = tree['parent'].min()
    return _recurse_leaf_dfs(tree, root)

In [25]:
%time app.view.ctree_view._whole_tree[app.view.ctree_view._whole_tree['parent'] == 17561]['child']

CPU times: user 227 µs, sys: 62 µs, total: 289 µs
Wall time: 209 µs


array([ 4697,  4704,  1805, ...,  8138, 17562, 17563])

In [ ]:
%time _get_leaves(app.view.ctree_view._whole_tree,17561)

In [29]:
app.view.ctree_view._clu_tree

array([(17561, 17562,  17.37668241,   139),
       (17561, 17563,  17.37668241, 16309),
       (17563, 17564,  23.23376301,   282),
       (17563, 17565,  23.23376301, 14754),
       (17565, 17566,  30.58011108,    46),
       (17565, 17567,  30.58011108, 11960),
       (17567, 17568,  31.53312042,   141),
       (17567, 17569,  31.53312042, 11376),
       (17569, 17570,  33.27811695,    30),
       (17569, 17571,  33.27811695, 10500),
       (17571, 17572,  33.90756386,   116),
       (17571, 17573,  33.90756386, 10090),
       (17573, 17574,  34.60248106,   365),
       (17573, 17575,  34.60248106,  9405),
       (17575, 17576,  41.23590468,    98),
       (17575, 17577,  41.23590468,  6558),
       (17577, 17578,  42.07801402,  6135),
       (17577, 17579,  42.07801402,    99),
       (17578, 17580,  46.68117365,    46),
       (17578, 17581,  46.68117365,  4562),
       (17581, 17582,  49.82471929,   341),
       (17581, 17583,  49.82471929,  3315),
       (17583, 17584,  51.305401

In [89]:
children = app.view.ctree_view._whole_tree[app.view.ctree_view._whole_tree['parent'] == 17561]
len(children[children['child_size'] == 1]['child'])

1113

In [90]:
def _get_leaves_2(tree, root):
    clu_tree = app.view.ctree_view._get_descendants(app.view.ctree_view._clu_tree, root)
    #print clu_tree
    chs = np.array([], dtype=np.int64)
    for node in clu_tree:
        children = tree[tree['parent'] == node]
        chs = np.append(chs, children[children['child_size'] == 1]['child'])
    return chs

In [49]:
_get_leaves_2(app.view.ctree_view._whole_tree, 17561)

[17561 17562 17563 17564 17565 17566 17567 17568 17569 17570 17571 17572
 17573 17574 17575 17576 17577 17578 17580 17581 17582 17583 17584 17586
 17587 17585 17579]


array([4697, 4704, 1805, ..., 4225, 4884, 8138])

In [97]:
%time app.view.ctree_view._get_leaves(app.view.ctree_view._whole_tree)
#r2 = _get_leaves_2(app.view.ctree_view._whole_tree, 17561)

CPU times: user 680 ms, sys: 12.7 ms, total: 692 ms
Wall time: 693 ms


array([4697, 4704, 1805, ..., 6321, 1515, 2736])

In [98]:
%time _get_leaves_2(app.view.ctree_view._whole_tree, 17561)

CPU times: user 2.41 ms, sys: 418 µs, total: 2.83 ms
Wall time: 2.39 ms


array([4697, 4704, 1805, ..., 6321, 1515, 2736])

In [107]:
(app.view.ctree_view._get_leaves(app.view.ctree_view._whole_tree, 17567) == _get_leaves_2(app.view.ctree_view._whole_tree, 17567)).all()

True

In [102]:
_get_leaves_2(app.view.ctree_view._whole_tree, 17563)

array([ 9524, 12989,  7356, ...,  6321,  1515,  2736])

In [96]:
f

True

In [94]:
np.sort(r2)

array([    0,     1,     2, ..., 17558, 17559, 17560])

In [ ]:
before = app.model.clu[10].membership.copy()

In [ ]:
np.unique(app.model.clu[10].membership)

In [ ]:
np.unique(app.model.clu[11].membership)

In [ ]:
app.model.clu[11].membership[app.view.ctree_view._get_leaves(app.view.ctree_view._whole_tree,9692)]

In [ ]:
app.model.clu[11].membership[app.view.ctree_view._get_leaves(app.view.ctree_view._whole_tree,9693)]

In [ ]:
app.view.ctree_view._select_clusters

In [ ]:
app.view.ctree_view._get_children(app.view.ctree_view._clu_tree, 15222)

In [ ]:
app.model.clu[6].index_id

In [ ]:
app.model.clu[6].index_id

In [ ]:
from hdbscan import HDBSCAN

In [ ]:
hdbcluster = HDBSCAN(min_cluster_size=18, 
                     leaf_size=20,
                     gen_min_span_tree=False,
                     algorithm='boruvka_kdtree')

In [ ]:
clusterer = hdbcluster.fit(app.model.fet[12])

**下面这个图显然选的不是面积最大的cluster，好奇怪，哪里不对呢**

In [ ]:
clusterer.condensed_tree_.plot(select_clusters=True,
                               selection_palette=sns.color_palette('deep', 8), log_size=True)

In [ ]:
clusterer.labels_

In [ ]:
sc = np.array(app.view.ctree_view._select_clusters, dtype=np.int64)
sc

In [ ]:
np.delete(sc, 1)

In [ ]:
np.sort(np.append(np.delete(sc, np.where(sc==7453)[0]),sc))

In [ ]:
#### app.model.tofile('/Users/chiy/Documents/HHMI/Github/spiketag_test_data/ttd_6bb4_25k_20170712_without_noise_spktag.bin')

In [ ]:
app.view.ctree_view._select_clusters

In [ ]:
idx = app.view.ctree_view._get_leaves(app.view.ctree_view._whole_tree, 15222)
idx

In [ ]:
app.view.ctree_view._get_leaves(app.view.ctree_view._clu_tree)

In [ ]:
app.model.clu[6].select(idx)

In [ ]:
np.unique(app.model.clu[6].membership)

### Detect peak and output pivotal file

In [ ]:
bf = bload(nCh=64, fs=25000)

In [ ]:
bf.load('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/tt16bit_160.bin', dtype='int16')

In [ ]:
spks = bf.detect_spks()

In [ ]:
data = bf.asarray()

In [ ]:
from ipywidgets import interact

In [ ]:
@interact(x=(1000,55752704,100), y=(0,63,1))
def plot_wave(x,y):
    plot(data[x-1000:x,y])

In [ ]:
from spiketag.view import wave_view

In [ ]:
data = bf.npmm.reshape(-1,64).astype(np.float32)

In [ ]:
wv = wave_view(data=data, spks=spks)

In [ ]:
wv.show()

In [ ]:
dtree_in_clu = app.model.clu[11].dtree
dtree_in_clu.shape

In [ ]:
app.view.dtree_view.set_data(app.model.clu[11])

In [ ]:
clu_tree = app.view.dtree_view._clu_tree.copy()
clu_tree.shape

In [ ]:
app.view.dtree_view._vertices

In [ ]:
app.view.dtree_view._vertices.shape[0] / 4.0

In [ ]:
last_leaf, root = clu_tree['parent'].max(), clu_tree['parent'].min()

In [ ]:
s = 0
for c in range(last_leaf, root-1, -1):
    s += clu_tree[clu_tree['parent'] == c].shape[0]
s

In [ ]:
clu_tree[clu_tree['parent'] == 9691]['child_size']

In [ ]:
sum(clu_tree['child_size'] > 1)

In [ ]:
a = np.array([1,2,3,4])
b = np.array([3,4])

In [ ]:
(a[-2:] == b).all()

In [ ]:
for g,c in zip(np.array([[1,2,3],[4,5,6]]),np.array([[10],[11]])):
    print g,c

In [ ]:
a = np.array([1,2])

In [ ]:
np.append(a, b)